In [16]:
import numpy as np
from matrepr import mdisplay, mprint

####seção hard coded##############################################
# o transformador está entre 3 e 4, logo k=3 e m=4
ykm = -5j
omega = 30;
a = 1;
omega_rad = omega * np.pi / 180;
###############################################################################

def v_curto(barra,Y,debugg = 0):
    #Matriz admitancia com os valores nas barras dos transformadores ajustadas de acordo
    Y_ajustada  = Y
    """#Y_ajustada[k,m]
    Y_ajustada[2,2] = (a**2 * np.exp(-1j*omega_rad))*ykm  #Ykk
    Y_ajustada[2,3] = (-a * np.exp(-1j*omega_rad))*ykm    #Ykm
    Y_ajustada[3,3] = (np.exp(-1j*omega_rad))*ykm         #Ymm
    Y_ajustada[3,2] = (-a * np.exp(1j*omega_rad))*ykm     #Ymk
    """
    Z_real = Y**-1
    #mdisplay(Z_real)
    Z_real = np.nan_to_num(Z_real, copy=False)
    #mdisplay(Z_real)

    # Cria um layout do sistema Matriz_layout
    Matriz_layout = (Y != 0).astype(int)
    #mdisplay(Matriz_layout)

    Z = np.linalg.inv(Y)

    #mdisplay(Y, 'latex')
    #mdisplay(Y_ajustada, 'latex')
    #mdisplay(Z, 'latex')
    n_barras = Y.shape[0]

    Z_n = Z[barra-1,barra-1]
    #corrente de curto na barra n (curto trifásico para o terra), assumindo tensão nominal na barra antes do curto (1 pu)
    Icn = (1)/Z_n
    #print(Icn)
    I_matriz_curto_barra_n = np.zeros((n_barras,1), dtype=complex)
    I_matriz_curto_barra_n[barra-1] = -Icn
    v_delta_n = Z @ I_matriz_curto_barra_n.flatten()  # Direct matrix multiplication

    #mprint(v_delta_n)
    v_iniciais = np.ones((n_barras,1)) #é assumindo que as tensões iniciais nas barras é 1pu
    v_n = v_delta_n + v_iniciais.flatten()
    # Trunca valores muito pequenos para prevenir erros de floating point
    v_n[np.abs(v_n) < 1e-10] = 0
    #mdisplay(v_n)

    print(f"Com o curto na barra {barra}, temos as tensões nas barras como:")
    for i in range(len(v_delta_n)):
        print(f"V_barra{i+1} = {v_n[i]:.4f} p.u.")

    #o calculo da correntes deve ser feito a mão pois depende do layout da rede
    # I=V/Z

    I_sistema = np.zeros((n_barras,n_barras), dtype=complex)
    for i in range(n_barras):
      for j in range(n_barras):
        if i != j and Matriz_layout[i, j] != 0: # Only calculate current if connected
          I_sistema[i,j] = (v_n[i] - v_n[j])/Z_real[i,j]
    I_sistema[np.abs(I_sistema) < 1e-10] = 0
    I_fluxo = I_sistema * Matriz_layout


    print("\nFluxo de Corrente:")
    for i in range(n_barras):
        for j in range(n_barras):
            if abs(I_fluxo[i, j]) > 1e-10:  # Check for non-zero current flow
                print(f"A corrente {I_fluxo[i, j]:.4f} p.u. flui da barra {i+1} para a barra {j+1}")

#######################################################################################################Debug
#conferir as conexões determinadas pela matriz de layout
    if debugg:
      print("\nConexoes:")
      for i in range(n_barras):
          for j in range(n_barras):
            if abs(Matriz_layout[i, j]) > 0:  # Check for non-zero current flow
              print(f"a barra {i+1} se conecta a barra {j+1}")
      print("\nmatriz de layout")
      mdisplay(Matriz_layout)
      print("\n matriz Y crua")
      mdisplay(Y)
      print("\n matriz Y considerando o transformador")
      mdisplay(Y_ajustada)
      print("\n matriz Z calculada a partir de Y , Z=Y^-1")
      mdisplay(Z)
      print(f"\n Z da barra {barra} em curto para o terra é de Z_n {Z_n} e a currente de curto é de {Icn}")
      print("\n Z real obtida do sistema")
      mdisplay(Z_real)
      print("\n v_delta_n")
      mprint(v_delta_n)
      print("\n v_n")
      mprint(v_n)
      print("Correntes no sistema:")
      mdisplay(I_sistema)
      print("Correntes no fluxo:")
      mdisplay(I_fluxo)



In [14]:
import numpy as np
from matrepr import mdisplay, mprint

# ====================================================================
# SEÇÃO HARD CODED - Parâmetros do Transformador
# ====================================================================
# O transformador está entre as barras 3 e 4, logo k=3 e m=4
ykm = -5j
omega = 30
a = 1
omega_rad = omega * np.pi / 180

# ====================================================================
# FUNÇÃO PRINCIPAL - Análise de Curto-Circuito
# ====================================================================
def v_curto(barra, Y, debugg=0):
    """
    Calcula tensões e correntes durante curto-circuito em uma barra específica
    
    Parâmetros:
    - barra: número da barra onde ocorre o curto (base 1)
    - Y: matriz admitância do sistema
    - debugg: flag para mostrar informações detalhadas de debug
    """
    
    # ----------------------------------------------------------------
    # PREPARAÇÃO DAS MATRIZES
    # ----------------------------------------------------------------
    Y_ajustada = Y  # Matriz admitância ajustada (transformador)
    
    # Código comentado para ajustes do transformador:
    """
    Y_ajustada[2,2] = (a**2 * np.exp(-1j*omega_rad))*ykm  #Ykk
    Y_ajustada[2,3] = (-a * np.exp(-1j*omega_rad))*ykm    #Ykm
    Y_ajustada[3,3] = (np.exp(-1j*omega_rad))*ykm         #Ymm
    Y_ajustada[3,2] = (-a * np.exp(1j*omega_rad))*ykm     #Ymk
    """
    
    # Cálculo das matrizes impedância
    Z_real = Y**-1  # Impedâncias individuais dos elementos
    Z_real = np.nan_to_num(Z_real, copy=False)
    
    Z = np.linalg.inv(Y)  # Matriz impedância do sistema
    
    # Layout do sistema (conexões entre barras)
    Matriz_layout = (Y != 0).astype(int)
    n_barras = Y.shape[0]
    
    # ----------------------------------------------------------------
    # CÁLCULO DO CURTO-CIRCUITO
    # ----------------------------------------------------------------
    # Impedância da barra em curto
    Z_n = Z[barra-1, barra-1]
    
    # Corrente de curto trifásico (assumindo 1 pu antes do curto)
    Icn = 1 / Z_n
    
    # Matriz de corrente de curto
    I_matriz_curto_barra_n = np.zeros((n_barras, 1), dtype=complex)
    I_matriz_curto_barra_n[barra-1] = -Icn
    
    # ----------------------------------------------------------------
    # CÁLCULO DAS TENSÕES DURANTE O CURTO
    # ----------------------------------------------------------------
    # Variação de tensão devido ao curto
    v_delta_n = Z @ I_matriz_curto_barra_n.flatten()
    
    # Tensões iniciais (1 pu em todas as barras)
    v_iniciais = np.ones((n_barras, 1))
    
    # Tensões finais após o curto
    v_n = v_delta_n + v_iniciais.flatten()
    
    # Trunca valores muito pequenos (erro de floating point)
    v_n[np.abs(v_n) < 1e-10] = 0
    
    # ----------------------------------------------------------------
    # EXIBIÇÃO DOS RESULTADOS - TENSÕES
    # ----------------------------------------------------------------
    print(f"Com o curto na barra {barra}, temos as tensões nas barras como:")
    for i in range(len(v_delta_n)):
        print(f"V_barra{i+1} = {v_n[i]:.4f} p.u.")
    
    # ----------------------------------------------------------------
    # CÁLCULO DAS CORRENTES NO SISTEMA
    # ----------------------------------------------------------------
    # O cálculo das correntes deve ser feito manualmente (depende do layout)
    I_sistema = np.zeros((n_barras, n_barras), dtype=complex)
    
    for i in range(n_barras):
        for j in range(n_barras):
            if i != j and Matriz_layout[i, j] != 0:  # Só calcula se conectado
                I_sistema[i, j] = (v_n[i] - v_n[j]) / Z_real[i, j]
    
    # Trunca correntes muito pequenas
    I_sistema[np.abs(I_sistema) < 1e-10] = 0
    I_fluxo = I_sistema * Matriz_layout
    
    # ----------------------------------------------------------------
    # EXIBIÇÃO DOS RESULTADOS - CORRENTES
    # ----------------------------------------------------------------
    print("\nFluxo de Corrente:")
    for i in range(n_barras):
        for j in range(n_barras):
            if abs(I_fluxo[i, j]) > 1e-10:
                print(f"A corrente {I_fluxo[i, j]:.4f} p.u. flui da barra {i+1} para a barra {j+1}")
    
    # ----------------------------------------------------------------
    # SEÇÃO DE DEBUG (INFORMAÇÕES DETALHADAS)
    # ----------------------------------------------------------------
    if debugg:
        print("\n" + "="*50)
        print("INFORMAÇÕES DE DEBUG")
        print("="*50)
        
        # Conexões do sistema
        print("\nConexões:")
        for i in range(n_barras):
            for j in range(n_barras):
                if abs(Matriz_layout[i, j]) > 0:
                    print(f"A barra {i+1} se conecta à barra {j+1}")
        
        # Matrizes do sistema
        print("\nMatriz de layout:")
        mdisplay(Matriz_layout)
        
        print("\nMatriz Y original:")
        mdisplay(Y)
        
        print("\nMatriz Y ajustada (transformador):")
        mdisplay(Y_ajustada)
        
        print("\nMatriz Z (Z = Y^-1):")
        mdisplay(Z)
        
        print(f"\nImpedância da barra {barra}: Z_n = {Z_n}")
        print(f"Corrente de curto: {Icn}")
        
        print("\nMatriz Z_real (impedâncias individuais):")
        mdisplay(Z_real)
        
        print("\nVariação de tensão (v_delta_n):")
        mprint(v_delta_n)
        
        print("\nTensões finais (v_n):")
        mprint(v_n)
        
        print("\nMatriz de correntes do sistema:")
        mdisplay(I_sistema)
        
        print("\nMatriz de fluxo de correntes:")
        mdisplay(I_fluxo)

In [17]:
"""Y_entrada = np.array([
    [-10.84j,   3.174j,  1j,      0,       0,       0,       0],
    [  3.174j, -3.174j,  0,       0,       0,       0,       0],
    [   1j,      0,     -6j,     5j,       0,       0,       0],
    [   0,       0,      5j,   -5.266j,  0.095j,  0.076j,  0.095j],
    [   0,       0,      0,     0.095j, -0.095j,    0,       0],
    [   0,       0,      0,     0.076j,    0,    -0.076j,    0],
    [   0,       0,      0,     0.095j,    0,       0,    -5.095j]
], dtype=complex)"""

Y_entrada = np.array([
    [-12.33j,    0,       4j,      5j],
    [   0,    -10.83j,    2.5j,    5j],
    [   4j,      2.5j,  -17.83j,   8j],
    [   5j,      5j,      8j,    -18j],
], dtype=complex)

v_curto(4,Y_entrada,1)

Com o curto na barra 4, temos as tensões nas barras como:
V_barra1 = 0.3754+0.0000j p.u.
V_barra2 = 0.3824+0.0000j p.u.
V_barra3 = 0.3246+0.0000j p.u.
V_barra4 = 0.0000+0.0000j p.u.

Fluxo de Corrente:
A corrente -0.0000+0.2031j p.u. flui da barra 1 para a barra 3
A corrente -0.0000+1.8769j p.u. flui da barra 1 para a barra 4
A corrente -0.0000+0.1445j p.u. flui da barra 2 para a barra 3
A corrente -0.0000+1.9120j p.u. flui da barra 2 para a barra 4
A corrente 0.0000-0.2031j p.u. flui da barra 3 para a barra 1
A corrente 0.0000-0.1445j p.u. flui da barra 3 para a barra 2
A corrente -0.0000+2.5968j p.u. flui da barra 3 para a barra 4
A corrente 0.0000-1.8769j p.u. flui da barra 4 para a barra 1
A corrente 0.0000-1.9120j p.u. flui da barra 4 para a barra 2
A corrente 0.0000-2.5968j p.u. flui da barra 4 para a barra 3

Conexoes:
a barra 1 se conecta a barra 1
a barra 1 se conecta a barra 3
a barra 1 se conecta a barra 4
a barra 2 se conecta a barra 2
a barra 2 se conecta a barra 3
a barra

C:\Users\jooj\AppData\Local\Temp\ipykernel_17716\3998964276.py:21: RuntimeWarning: invalid value encountered in reciprocal
  Z_real = Y**-1


,0,1,2,3
0,1,0,1,1
1,0,1,1,1
2,1,1,1,1
3,1,1,1,1



 matriz Y crua


,0,1,2,3
0,-0--12.33i,0+0i,0+4i,0+5i
1,0+0i,-0--10.83i,0+2.5i,0+5i
2,0+4i,0+2.5i,-0--17.83i,0+8i
3,0+5i,0+5i,0+8i,-0--18i


...

,0,1,2,3
0,-0+0.0811i,0+0i,0--0.25i,0--0.2i
1,0+0i,-0+0.09234i,0--0.4i,0--0.2i
2,0--0.25i,0--0.4i,-0+0.05609i,0--0.125i
3,0--0.2i,0--0.2i,0--0.125i,-0+0.05556i



 v_delta_n
<length=4, 4 'complex128' elements, array>
     0           1           2         3
[-0.6246+0i  -0.6176+0i  -0.6754+0i  -1+0i]

 v_n
<length=4, 4 'complex128' elements, array>
     0          1          2       3
[0.3754+0i  0.3824+0i  0.3246+0i  0+0i]
Correntes no sistema:


,0,1,2,3
0,0+0i,0+0i,-0+0.2031i,-0+1.877i
1,0+0i,0+0i,-0+0.1445i,-0+1.912i
2,-0--0.2031i,-0--0.1445i,0+0i,-0+2.597i
3,-0--1.877i,-0--1.912i,-0--2.597i,0+0i


Correntes no fluxo:


,0,1,2,3
0,0+0i,0+0i,-0+0.2031i,-0+1.877i
1,0+0i,0+0i,-0+0.1445i,-0+1.912i
2,0--0.2031i,0--0.1445i,0+0i,-0+2.597i
3,0--1.877i,0--1.912i,0--2.597i,0+0i
